**A. Exploratory Data Analysis EDA**

**Pour Bien Comprendre nos Datasets (NEWS & MARKET) , et découvrire la structure des données , on va utiliser l'approche EDA.**

***1.Importer les Librairies et acquérir les données:***

In [ ]:
import numpy as np 
import pandas as pd 
from itertools import chain
import os
print(os.listdir("../input"))

In [ ]:
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()

On a deux datasets , on va traiter chacune à part

***2.EDA pour Market Data:***

In [ ]:
market_train_df.head()

In [ ]:
market_train_df.tail()

In [ ]:
market_train_df.shape

Alors on constate que Market dataset contient 4072956 lignes et 16 colonnes. mais quelles sont ces colonnes?

In [ ]:
market_train_df.columns

Maintenant , on a une idée sur les noms des colonnes , il reste qu'afficher le type de chaque colonne

In [ ]:
market_train_df.info()

Et voila , on trouve qu'il ya trois colonnes qui sont pas des float , c'est: time , AssetCode, et AssetName.
Maintenant on peut extraire quelques statistiques pour ces données:

In [ ]:
market_train_df.describe()

D'abord , on a une idée générale sur nos données , il reste à decouvrire s'il ya des valeurs nulles dans le dataset ou pas:

In [ ]:
market_train_df.isnull().any()

On a 4 colonnes contenant les valeurs nulles, on va decouvrire le pourcentage de ces valeurs dans notre dataset et puis les visualiser:

In [ ]:
market_train_df.isnull().sum()*100/market_train_df.shape[0]

In [ ]:
import matplotlib.pyplot as plt
percent = (market_train_df.isnull().sum()*100/market_train_df.shape[0]).sort_values(ascending=False)
percent.plot(kind="bar", figsize = (20,10), fontsize = 20)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Value Percent(%)", fontsize = 20)
plt.title("Total Missing Value by market_obs_df", fontsize = 20)


Les colonnes returnsClosePrevMktres1 , returnsOpenPrevMktres1,returnsClosePrevMktres10,returnsOpenPrevMktres1 ont des valeurs nulles , ces colonnes refèrent au Residual returns qui se calcule par la formule suivante:**mktres=raw−β*rmarket** .
Dans notre cas on peut pas calculer le β ,donc on va remplacer ces valeurs nulles par la médiane ou bien la moyenne , ça depends des outiliers qu'on va les visualiser tout de suite:


In [ ]:
plt.figure(figsize=(10,10))
market_train_df['returnsClosePrevMktres1'].plot(kind='box')

Ici On a un nombre faible des outliers , donc on va savoir la distribution par rapport à la médiane et la moyenne:

![](http://)Par rapport à la moyenne:

In [ ]:
( market_train_df['returnsClosePrevMktres1']<market_train_df['returnsClosePrevMktres1'].mean()).value_counts()

Par rapport à la médiane:

In [ ]:
( market_train_df['returnsClosePrevMktres1']<market_train_df['returnsClosePrevMktres1'].median()).value_counts()

* Donc on va remplacer par la moyenne:

In [ ]:
market_train_df['returnsClosePrevMktres1'].fillna(market_train_df['returnsClosePrevMktres1'].mean(),inplace=True)

In [ ]:
market_train_df.isnull().any()

Passant maintenant au autres colonnes contenant les valeurs nulles:

In [ ]:
market_train_df['returnsOpenPrevMktres1'].plot.box()

In [ ]:
market_train_df['returnsClosePrevMktres10'].plot.box()

In [ ]:
market_train_df['returnsOpenPrevMktres10'].plot.box()

On a plusieurs ouliers , donc on va remplacer les valeurs nulles par la médiane:

In [ ]:
market_train_df['returnsOpenPrevMktres1'].fillna(market_train_df['returnsOpenPrevMktres1'].median(),inplace=True)
market_train_df['returnsClosePrevMktres10'].fillna(market_train_df['returnsClosePrevMktres10'].median(),inplace=True)
market_train_df['returnsOpenPrevMktres10'].fillna(market_train_df['returnsOpenPrevMktres10'].median(),inplace=True)

In [ ]:
market_train_df.isnull().any()

Maintenant on a traité les valeurs nulles de Market Data , passant à l'exploration du News Data.

***3.EDA News Data***

In [ ]:
news_train_df.head()

In [ ]:
news_train_df.tail()

In [ ]:
news_train_df.shape

News dataset contient 9328750 ligne et 35 colonne.

In [ ]:
news_train_df.columns

In [ ]:
news_train_df.info()

In [ ]:
news_train_df.describe()

In [ ]:
news_train_df.isnull().any()

alors on a pas de valeurs manquantes dans news dataset.

Maintenant on va retourner vers Market dataset et visualiser les prix des quelques  assetcodes 

In [ ]:
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
data = []
for asset in np.random.choice(market_train_df['assetCode'].unique(), 10):
    asset_df = market_train_df[(market_train_df['assetCode'] == asset)]

    data.append(go.Scatter(
        x = asset_df['time'].dt.strftime(date_format='%Y-%m-%d').values,
        y = asset_df['close'].values,
        name = asset
    ))
layout = go.Layout(dict(title = "Closing prices of 10 random assets",
                  xaxis = dict(title = 'Month'),
                  yaxis = dict(title = 'Price (USD)'),
                  ),legend=dict(
                orientation="h"))
py.iplot(dict(data=data, layout=layout), filename='basic-line')

Normalement on a les données de 2007 jusqu'a maintenant , Or dans 2008 il y avait une crise mondiale s'appellle  la crise bancaire et financière de l'automne 2008 , donc on va jeter un coup d'eil sur les données de Market dans cette période:********

In [ ]:
market_train_df['price_diff'] = market_train_df['close'] - market_train_df['open']
market_train_df.sort_values('price_diff')[:10]

Donc, le prix des actions "Towers Watson & Co" et Bank of New York Mellon Corp  était presque 10k ... ce qui est anormale .
Maintenant, on pense qu'il est temps de jeter une vieille partie du dataset. Laissons seulement les données depuis l'année 2009 , pour se  débarrasser des données de la plus grande crise.

In [ ]:
news_train_df = news_train_df.loc[news_train_df['time'] >= '2009-01-01 22:00:00+0000']
market_train_df = market_train_df.loc[market_train_df['time'] >= '2009-01-01 22:00:00+0000']

**B.Features Engineering**

Dans l'approche Features Engineering , on essaye de garder les caractéristiques les plus pertinents et les plus utils dans notre étude , pour cela on va dans un premier moment , on va faire une étude de corrélation pour les données séparés , puis on va faire une jointure des deux tables (Market et News) et appliquer des modèles afin de savoir l'importance des Caractéristiques .

***1.Market Dataset***

On va savoir la corrélation entre les données du Market premièrement et puis la corrélation entre les caractéristiques et la cible

In [ ]:
corr=market_train_df.corr()
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(corr,cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,len(corr.columns),1)
ax.set_xticks(ticks)
plt.xticks(rotation=90)
ax.set_yticks(ticks)
ax.set_xticklabels(corr.columns)
ax.set_yticklabels(corr.columns)
plt.show()

Alors on constate qu'il ya des caractéristiques qui ont une forte corrélation entre eux et il ya ceux qui ont une faible corrélation entre eux , mais l'essentiel pour nous est de savoir la corrélation entre la cible et les autres caractéristiques

In [ ]:
corr_with_mkt = market_train_df.corr()["returnsOpenNextMktres10"].sort_values(ascending=False)
plt.figure(figsize=(14,7))
corr_with_mkt.drop("returnsOpenNextMktres10").plot.bar()
plt.show()

***2.News Dataset***

In [ ]:
corr2=news_train_df.corr()
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(corr2,cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,len(corr2.columns),1)
ax.set_xticks(ticks)
plt.xticks(rotation=90)
ax.set_yticks(ticks)
ax.set_xticklabels(corr2.columns)
ax.set_yticklabels(corr2.columns)
plt.show()

Maintenant on veux savoir la corrélation entre les caractéristiques du news data et le sentiments class:

In [ ]:
corr_with_mkt = news_train_df.corr()["sentimentClass"].sort_values(ascending=False)
plt.figure(figsize=(14,7))
corr_with_mkt.drop("sentimentClass").plot.bar()
plt.show()

Maintenant , on va faire une jointure de données , et savoir l'importance de chaque caractéristique pour le returnsOpenNextMktres10 .

Avant de faire la jointure , on va prendre que 1_500_000 ligne du market data et 3_500_000 du News data.

In [ ]:
market = market_train_df.head(1_500_000)
news = news_train_df.head(3_500_000)

In [ ]:
news_cols_agg = {
    'urgency': ['min', 'count'],
    'takeSequence': ['max'],
    'bodySize': ['min', 'max', 'mean'],
    'wordCount': ['min', 'max', 'mean'],
    'sentenceCount':['min', 'max', 'mean'],
    'companyCount': ['min', 'max', 'mean'],
    'relevance': ['min', 'max', 'mean'],
    'sentimentClass': ['min', 'max', 'mean'],
    'sentimentNegative': ['min', 'max', 'mean'],
    'sentimentNeutral': ['min', 'max', 'mean'],
    'sentimentPositive': ['min', 'max', 'mean'],
    'sentimentWordCount': ['min', 'max', 'mean'],
    'noveltyCount12H': ['min', 'max', 'mean'],
    'noveltyCount24H': ['min', 'max', 'mean'],
    'noveltyCount3D': ['min', 'max', 'mean'],
    'noveltyCount5D': ['min', 'max', 'mean'],
    'noveltyCount7D':['min', 'max', 'mean'],
    'volumeCounts12H': ['min', 'max', 'mean'],
    'volumeCounts24H': ['min', 'max', 'mean'],
    'volumeCounts3D': ['min', 'max', 'mean'],
    'volumeCounts5D': ['min', 'max', 'mean'],
    'volumeCounts7D': ['min', 'max', 'mean']
}

La fonction pour faire la jointure

In [ ]:
def merge_data(news_train_df,market_train_df):
    #rendre les assetCodes dans une liste
    news_train_df['assetCodes'] = news_train_df['assetCodes'].str.findall(f"'([\w\./]+)'")
    news_train_df.time= news_train_df.time.dt.date
    market_train_df.time= market_train_df.time.dt.date
    #faire sortir élément par élément de la liste
    assetCodes_expanded = list(chain(*news_train_df['assetCodes']))
    #création d'un array ayant les indexes répétés de chaque ligne dupliquée
    assetCodes_index = news_train_df.index.repeat( news_train_df['assetCodes'].apply(len) )
    #Création d'un dataframe contenant les assetCodes et leurs indexes
    df_assetCodes = pd.DataFrame({'level_0': assetCodes_index, 'assetCode': assetCodes_expanded})
    # Creation d'un news dataframe où les lignes sont répétées
    news_cols = ['time', 'assetCodes'] + sorted(news_cols_agg.keys())
    #je dois ajouter get dummies pour provider à ce niveau là
    news_train_df_expanded = pd.merge(df_assetCodes, news_train_df[news_cols], left_on='level_0',
                                  right_index=True, suffixes=(['','_old']))
     # Free memory
    del news_train_df, df_assetCodes
    news_train_df_aggregated = news_train_df_expanded.groupby(['time', 'assetCode']).agg(news_cols_agg)
     # Free memory
    del news_train_df_expanded
    # Convert to float32 to save memory
    news_train_df_aggregated = news_train_df_aggregated.apply(np.float32)
    # Flat columns
    news_train_df_aggregated.columns = ['_'.join(col).strip() for col in news_train_df_aggregated.columns.values]
    # Join with train
    market_train_df = market_train_df.join(news_train_df_aggregated, on=['time', 'assetCode'])
    # Free memory
    del news_train_df_aggregated
    return market_train_df

In [ ]:
data_m=merge_data(news,market)

On va se debarasser des valeurs nulles :

In [ ]:
data_m=data_m.dropna()
data_m.shape 

et voila on a obtenu 414779 lignes de données. Maintenant il reste à decouvrire la corrélation entre la cible et les autres colonnes

In [ ]:
corr_with_mkt1 = data_m.corr()["returnsOpenNextMktres10"].sort_values(ascending=False)
plt.figure(figsize=(14,7))
corr_with_mkt1.drop("returnsOpenNextMktres10").plot.bar()
plt.show()

Il ya des caractéristiques qui ont une forte corrélation avec la cible et il ya ceux qui ont une faible corrélation , maintenat on va appliquer quelques modèles pour savoir l'importance de ces caractéristiques , mais avant ça il faut préparer ces données

In [ ]:
Y=data_m.returnsOpenNextMktres10.values 
data_f=data_m.drop(['returnsOpenNextMktres10'],axis=1)
data_final=data_f.drop(['assetName'],axis=1) #On a deja l'assetCode 

In [ ]:
print(data_final.dtypes)

Alors il faut avoir le meme type (float64) pour tous les données

In [ ]:
from sklearn import preprocessing
del data_final['time']
le = preprocessing.LabelEncoder()
le.fit(data_final['assetCode'])
data_final['assetCode']=le.fit_transform(data_final['assetCode'])

In [ ]:
data_final=data_final.astype(float)

In [ ]:
print(data_final.dtypes)

Maintenant les données sont prets , il reste à appliquer les modèles 

In [ ]:
X=data_final.values

In [ ]:
from sklearn.preprocessing import MinMaxScaler,Imputer
# définir un dictionnaire pour stocker nos rankings 
ranks = {}
# créer une fonction qui stocke le classement des caractéristiques 
def ranking(ranks, names, order=1):
    minmax = MinMaxScaler()
    ranks = minmax.fit_transform(order*np.array([ranks]).T).T[0]
    ranks = map(lambda x: round(x,2), ranks)
    return dict(zip(names, ranks))

Parmis les algorithmes qu'on va appliquer , on trouve LinearRegression, Ridge, Lasso et LogisticRegression, puis on va extraire l'importance de chaque caractéristique pour chaque algorithme et faire la moyenne par la suite :

In [ ]:
from sklearn.feature_selection import RFE, f_regression
from sklearn.preprocessing import StandardScaler
X_scaled=StandardScaler().fit_transform(X)
from sklearn.linear_model import (LinearRegression, Ridge, Lasso,LogisticRegression)
colnames=data_final.columns
lr = LinearRegression(normalize=True)
lr.fit(X_scaled,Y)
ranks["LinReg"] = ranking(np.abs(lr.coef_), colnames)

#  Ridge 
ridge = Ridge(alpha = 7)
ridge.fit(X_scaled,Y)
ranks['Ridge'] = ranking(np.abs(ridge.coef_), colnames)

#  Lasso
lasso = Lasso(alpha=.05)
lasso.fit(X_scaled, Y)
ranks["Lasso"] = ranking(np.abs(lasso.coef_), colnames)

In [ ]:
r = {}
for name in colnames:
    r[name] = round(np.mean([ranks[method][name] 
                             for method in ranks.keys()]), 2)
 
methods = sorted(ranks.keys())
ranks["Mean"] = r
methods.append("Mean")
 
print("\t%s" % "\t".join(methods))
for name in colnames:
    print("%s\t%s" % (name, "\t".join(map(str, 
                         [ranks[method][name] for method in methods]))))

In [ ]:
# mettre la moyenne dans un dataframe Pandas
meanplot = pd.DataFrame(list(r.items()), columns= ['Feature','Mean Ranking'])

# Trier le  dataframe
meanplot = meanplot.sort_values('Mean Ranking', ascending=False)

In [ ]:
import seaborn as sns
sns.factorplot(x="Mean Ranking", y="Feature", data = meanplot, kind="bar", 
               size=20, aspect=1.9, palette='coolwarm')

Maintenant on va prendre que les caractéristiques qui ont une importance pour notre cible

In [ ]:
features=list(meanplot[meanplot['Mean Ranking']!=0].Feature)